# Generic Box
This code communicates with a component. The component was launched with the command below from the root of the repository:

```shell
$ docker run --rm -it -p 4061:8061 -v ./src/external1.py=/workspace/external.py generic
```


In [ ]:
import argparse
import grpc
import io
import matplotlib.pyplot as plt
from importlib.machinery import SourceFileLoader
image_generic_pb2 = SourceFileLoader("image_generic_pb2","../protos/image_generic_pb2.py").load_module()
image_generic_pb2_grpc = SourceFileLoader("image_generic_pb2_grpc","../protos/image_generic_pb2_grpc.py").load_module()
#import image_generic_pb2
#import image_generic_pb2_grpc
import PIL.Image as PIL_image
import numpy as np
import cv2
from scipy.io import loadmat,savemat

In [ ]:
def process_image(stub, image_path):
    print(f'Processing image: \'{image_path}\'')
    with open(image_path, 'rb') as fp:
        image_bytes = fp.read()
    request = image_generic_pb2.Image(data=image_bytes)
    return stub.process(request)


def display_image(image):
    if isinstance(image,np.ndarray):
        img =image
    else:
        img = PIL_image.open(io.BytesIO(image.data))
    ax = plt.gca()
    ax.imshow(img)
    plt.show()

def plot_features(img,kp):
    for p in kp:
        cv2.circle(img,(int(p[0]),int(p[1])), 3, (0,0,255), -1)
    ax =plt.gca()
    ax.imshow(img)
    plt.show()

def parse_args():
    """
    Parse arguments for test setup

    Returns:
        The arguments for the test
    """
    parser = argparse.ArgumentParser(description='Test for OpenPose gRPC Service')
    parser.add_argument(
        'image',
        help='Path to the image to send to the server')
    parser.add_argument(
        '--target',
        metavar='target',
        default='sipgpu1.isrnet:4061',
        help='Location of the tested server (defaults to localhost:8061)')
    return parser.parse_args()

## Set parameters (GRPC) and data
Set IP and port of the service (the same used in docker run), input data (image in a matfile) and call the service

In [ ]:
target='sipgpu1.isrnet:4061'
image_path='../images/eiffel.mat'
#Para imagens muito grandes buffer grande
channel_opt = [('grpc.max_send_message_length', 512 * 1024 * 1024), ('grpc.max_receive_message_length', 512 * 1024 * 1024)]
with grpc.insecure_channel(target,options=channel_opt) as channel:
    estimator_stub = image_generic_pb2_grpc.ImageGenericServiceStub(channel)
    #response = process_image(estimator_stub, image_path)
    with open(image_path, 'rb') as fp:
        image_bytes = fp.read()
    request = image_generic_pb2.Image(data=image_bytes)
    response=estimator_stub.process(request)

# Check output

In [ ]:
f=open("../images/output2.mat","wb")
f.write(response.data)
f.close()

In [ ]:
mat_data=loadmat("../images/output2.mat")
display_image(mat_data["im"])

For a python script include the cell below generate a .py file (instead of ipynb)

In [ ]:
if __name__ == '__main__':
    args = parse_args()
    target = args.target
    image_path = args.image
    with grpc.insecure_channel(target) as channel:
        estimator_stub = image_generic_pb2_grpc.ImageGenericServiceStub(channel)
        try:
            response = process_image(estimator_stub, image_path)
            display_image(response)
        except grpc.RpcError as rpc_error:
            print('An error has occurred:')
            print(f'  Error Code: {rpc_error.code()}')
            print(f'  Details: {rpc_error.details()}')